In [1]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

from sklearn import metrics
from sklearn.metrics import roc_auc_score

# import matplotlib.pyplot as plt

from sklearn.externals import joblib

In [2]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
# from hyperopt.mongoexp import MongoTrials

In [3]:
import matplotlib.pyplot as plt

In [4]:
from sklearn.metrics import accuracy_score

In [5]:
import pandas as pd
import numpy as np

In [6]:
import pickle

In [7]:
import time

### Model trainig with hyper-parameter tuning

In [8]:
## Model developed to be used in for loop without print statement

def model_fit_hyperopt(params):
    
#     attempt_count += 1

    ### Asssigning default parameters
#     useTrainCV = False
    cv_folds = 5
    early_stopping_rounds = 50
    
    ### Forming copy of input datasets
    train_c, valid_c = train_sample.copy(), valid_sample.copy()
#     out_path = params['out_path']
    predictors, target = df_features, df_target
    
    alg = XGBClassifier(
                learning_rate =params['learning_rate'],
                n_estimators=5000,
                max_depth=params['max_depth'],
                min_child_weight=params['min_child_weight'],
                gamma=params['gamma'],
                subsample=params['subsample'],
                colsample_bytree=params['colsample_bytree'],
                objective= 'binary:logistic',
                scale_pos_weight=params['scale_pos_weight'],
                seed=27, n_jobs=7)

#     if useTrainCV:
#         xgb_param = alg.get_xgb_params()
#         xgtrain = xgb.DMatrix(train_c[predictors].values, label=train_c[target].values)
#         xgtest = xgb.DMatrix(valid_c[predictors].values)
#         cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
#             metrics = 'auc', early_stopping_rounds=early_stopping_rounds, verbose_eval=False, seed=27)
#         ne_new = n_estimators=cvresult.shape[0]
#         alg.set_params(n_estimators=cvresult.shape[0])
        
    #Fit the algorithm on the data
    eval_set = [(valid_c[predictors],valid_c[target])]
    alg.fit(train_c[predictors], train_c[target],eval_metric='auc', early_stopping_rounds=early_stopping_rounds, eval_set = eval_set, verbose = False)     
    
    valid_c.loc[:,'prob'] = alg.predict_proba(valid_c[predictors])[:,1]
#     test_c.loc[:,'prob'] = alg.predict_proba(test_c[predictors])[:,1]
    
    valid_auc = roc_auc_score(np.array(valid_c.loc[:,target]), np.array(valid_c.loc[:,'prob']))
#     test_auc = roc_auc_score(np.array(test_c.loc[:,target]), np.array(test_c.loc[:,'prob']))
    loss = 1 - valid_auc
    del train_c
    del valid_c
#     if out_path != None:
#         f = open(os.path.join(out_path, "hyperopt_logs.txt"), "a+")
#         f.write("Valid AUC: {} and Test AUC: {}, loss: {}".format(valid_auc,test_auc,loss))
#         f.close()

    global num
    f1 = open("status.txt", "a+")
    f1.write("num:{} , loss:{}, params: {} \n \n".format(num,loss, params))
    f1.close() 
    num = num +1
    
    print ("Valid AUC: {} and Loss: {}".format(valid_auc, loss))
    
    return {'loss': loss, 'status': STATUS_OK, 'params': params}

In [9]:
def get_best_model_xgb(space):
    
#     f = open(os.path.join(out_path, "hyperopt_logs.txt"), "w+")
#     f.close()

    
    trials=Trials()
    global num
    num = 1
    best = fmin(model_fit_hyperopt, space, algo=tpe.suggest, max_evals=50, trials=trials)
    
    return trials.best_trial['result']

In [10]:
def train_best_model(best_params):
    cv_folds = 5
    early_stopping_rounds = 50
    
    train_c, valid_c = train_sample.copy(), valid_sample.copy()

    predictors, target = df_features, df_target
    
    alg = XGBClassifier(
                learning_rate =best_params['learning_rate'],
                n_estimators=5000,
                max_depth=best_params['max_depth'],
                min_child_weight=best_params['min_child_weight'],
                gamma=best_params['gamma'],
                subsample=best_params['subsample'],
                colsample_bytree=best_params['colsample_bytree'],
                objective= 'binary:logistic',
                scale_pos_weight=best_params['scale_pos_weight'],
                seed=27, n_jobs=7)
    
    eval_set = [(valid_c[predictors],valid_c[target])]
    alg.fit(train_c[predictors], train_c[target],eval_metric='auc', early_stopping_rounds=early_stopping_rounds, eval_set = eval_set, verbose = False)
    
    return alg

In [11]:
train_o = pd.read_pickle('../../data/train_insurance_2017.pkl')
valid_o = pd.read_pickle('../../data/valid_insurance_2017.pkl')
# test_o = pd.read_pickle('../../data/test_insurance_2017.pkl')

In [12]:
train_o.drop(['NewEstimateTotal'],axis=1,inplace=True)
valid_o.drop(['NewEstimateTotal'],axis=1,inplace=True)
# test_o.drop(['StartedFlag'],axis=1,inplace=True)

In [13]:
# train, valid, test, df_features, df_target = data(all_data,target='StartedFlag')
# del all_data
df_target = 'StartedFlag'
df_features = list(set(train_o.columns) - set([df_target]))
train_sample = train_o.head(100000)
valid_sample = valid_o.head(20000)
# del train
# del valid
del train_o
del valid_o
# del test_o
# del test

In [ ]:
start_time = time.time()
space = {'learning_rate': hp.choice('learning_rate',[0.01, 0.02, 0.04, 0.06, 0.08, 0.1, 0.2, 0.3, 0.4, 0.6, 0.8, 1]),
                     'min_child_weight': hp.choice('min_child_weight', [0,1,2,3,4,5,6,7]),
                     'max_depth': hp.choice('max_depth', [3,4,5,6,7,8,9,10]),
                     'gamma': hp.choice('gamma', [0,1,5,10]),
                     'subsample': hp.choice('subsample',np.arange(0.4,1.05,0.1)),
                     'colsample_bytree': hp.choice('colsample_bytree',np.arange(0.5,1.05,0.1)),
                     'scale_pos_weight': (len(train_sample) -train_sample[df_target].sum())/train_sample[df_target].sum(),
#                      'predictors': df_features,
#                      'target': df_target
        }

best_trails1 = get_best_model_xgb(space)

f = open("started_flag_100000_hyperopt_2017.txt", "w")
f.write("Took {} seconds with best trails as: {}".format(time.time()-start_time,best_trails1))
f.close() 

best_model = train_best_model(best_trails1['params'])

# To save model
with open('insurance_classify_100000_started_flag_hyperopt_2017.pkl', 'wb') as f:
    pickle.dump(best_model, f)

In [ ]:
# To load the pickle model\n",
with open('insurance_classify_100000_started_flag_hyperopt_2017.pkl', 'rb') as f:
    model = pickle.load(f)

In [16]:
# valid.loc[:,'prob_up'] = model.predict_proba(valid[df_features])[:,1]

# valid['prediction'] = np.where(valid.prob_up > 0.5,1,0)

# accuracy_score(valid.StartedFlag, valid.prediction)

In [17]:
# test.loc[:,'prob_up'] = model.predict_proba(test[df_features])[:,1]

# test['prediction'] = np.where(test.prob_up > 0.5,1,0)

# accuracy_score(test.StartedFlag, test.prediction)

# Making predictions on the Test data

In [21]:
test_o = pd.read_pickle('../../data/test_insurance_2017.pkl')
test_o.drop(['NewEstimateTotal'],axis=1,inplace=True)

In [22]:
test_o.shape

(414139, 2789)

In [25]:
test_new = pd.DataFrame()
for i,chunk in enumerate(np.array_split(test_o, 5)):
    print(i)
    pred = best_model.predict(chunk[df_features])
    chunk['preds'] = pred
    test_new = test_new.append(chunk)
test_new.to_pickle('test_insurance_2017_preds_started_flag_hyperopt.pkl')
acc = accuracy_score(test_new.StartedFlag, test_new.preds)

0
1
2
3
4


In [26]:
acc

0.805309328510476

In [30]:
# Confusion Matrix on test_new Data
metrics.confusion_matrix(test_new.StartedFlag, test_new.preds)

array([[156283,  45098],
       [ 35531, 177227]])

In [31]:
# Saving test csv result

In [32]:
def reverse_dummies(dummies_df):
    sep = '_'
    dummy_cols = list(set(col.split(sep)[0] for col in dummies_df.columns if sep in col))
    print(dummy_cols)
    other_cols = [col for col in dummies_df.columns if sep not in col]
    print(other_cols)
    dfs = []
    for i, col in enumerate(dummy_cols):
        print(i, col)
        dfs.append(dummies_df.filter(regex=col).rename(columns=lambda name: name.split(sep)[1]).idxmax(axis=1))

    df = pd.concat(dfs + [dummies_df[other_cols]], axis=1)
    df.columns = dummy_cols + other_cols
    return df

In [33]:
test_result = reverse_dummies(test_new)

['CommOrRes', 'DisplayName', 'State', 'DivisionName']
['StartedFlag', 'NoteCount', 'PhotoCount', 'JobCount', 'ClaimCount', 'LossYearMo', 'preds']
0 CommOrRes
1 DisplayName
2 State
3 DivisionName


In [35]:
test_result.to_csv('test_insurance_2017_preds_started_flag_hyperopt.csv')